In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [11]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 5, 30, 19, 7),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [17]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 14, 1, 43, 41))]


In [12]:
db.query('''SELECT TITLE, TARGET FROM POSTS;''')

[('AMC & GME Price Action Comparison before their squeeze', 1),
 ('GME & AMC', 1),
 ('7k Loss on GME 0DTE', -1),
 ('AMC GME Meme 🚀🌕', -1),
 ('Proper 401k weighting for a 26 year old? 99.8% $GME, 0.2% $AMC', -1),
 ('For all of you retarded apes talking about the $GME $AMC short squeezing to the millions',
  -1),
 ('Happy to see everyone make it all back on GME and AMC... Anyone still holding BB like a retard? Make',
  -1),
 ('Welp. I’ve been selling CC’s on my GME to bring my cost basis down since January. I knew this day wo',
  -1),
 ("I am an idiot. Wife doesn't know yet. Yolo'd into GME at the peak hoping to correct this loss.",
  -1),
 ('What if GME does a OSTK.', -1)]

In [39]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}' AND STOCK_ID='{}'".format(indexes[0][0], 'GME'))[0][0]

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [62]:
db.query("SELECT COUNT(*) FROM POSTS WHERE StOCK_ID='{}';".format("GME"))[0][0]

10

In [6]:
db = Database()
db.use_database('DB1')
#db.drop_table("LABELED_POSTS")
#db.drop_table("LABELED_COMMENTS")

db.initialize_tables()

scrapewsb = ScrapeWSB('GME', 10, 10)

df = scrapewsb.scrape()
scrapewsb.convert(df)
print("DONE")

connection established
[(datetime.datetime(2021, 5, 30, 19, 4, 20),)]
connection established
[(datetime.datetime(2021, 5, 30, 19, 4, 21),)]


  0%|          | 0/10 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 5, 30, 19, 4, 24),)]


100%|██████████| 10/10 [00:53<00:00,  5.33s/it]

DONE


In [4]:
db = Database()
db.use_database('DB1')

connection established
[(datetime.datetime(2021, 5, 30, 0, 20, 6),)]


In [8]:
print(db.query('show tables;'))

[('POSTS',)]


In [5]:
print(db.query('''SELECT POST_ID FROM POSTS WHERE TARGET=-1 ;'''))

[('nnpn9z',), ('nn4aej',), ('nns6mk',), ('nmpk6k',), ('nmjo2l',), ('nm53o9',), ('nmabst',), ('nmthag',), ('nmmu24',), ('nmpq6d',)]


In [4]:
print(db.query('''SHOW COLUMNS FROM LABELED_POSTS;'''))

[('POST_ID', b'char(20)', 'NO', '', None, ''), ('TARGET', b'int', 'YES', '', None, ''), ('TITLE', b'char(100)', 'YES', '', None, '')]


In [4]:
db.drop_table("LABELED_POSTS")
db.drop_table("LABELED_COMMENTS")

In [5]:
db.drop_table('COMMENTS')
db.drop_table("POSTS")
db.drop_table("STOCKS")

In [3]:
db.drop_table("POSTS")


ProgrammingError: 1051 (42S02): Unknown table 'DB1.POSTS'